In [2]:
# Импортируем наши библиотеки
import numpy as np  
import socket
from selenium import webdriver    
from bs4 import BeautifulSoup    
import requests    
import os
import pandas as pd 
pd.set_option('display.max_rows', 500)
from selenium.webdriver.support.ui import WebDriverWait as wait
from tqdm import tqdm
import random
import time
import pickle
from pandas_profiling import ProfileReport
from  pprint import pprint
import json
import traceback
from static_variables import resolve_static
import threading

PAGE_SELECTOR = '*CURRENT_PAGE*'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'}
regions_urls = ['https://auto.ru/moskva/cars/bmw/used/',
                'https://auto.ru/sankt-peterburg/cars/bmw/used/',
                'https://auto.ru/krasnodarskiy_kray/cars/bmw/used/',
                'https://auto.ru/rostovskaya_oblast/cars/bmw/used/',
                'https://auto.ru/sverdlovskaya_oblast/cars/bmw/used/',
                'https://auto.ru/tatarstan/cars/bmw/used/',
                'https://auto.ru/nizhegorodskaya_oblast/cars/bmw/used/',
                'https://auto.ru/chelyabinskaya_oblast/cars/bmw/used/',
                'https://auto.ru/samarskaya_oblast/cars/bmw/used/',
                'https://auto.ru/bashkortostan/cars/bmw/used/',
                'https://auto.ru/permskiy_kray/cars/bmw/used/',
                'https://auto.ru/krasnoyarskiy_kray/cars/bmw/used/',
                'https://auto.ru/voronezhskaya_oblast/cars/bmw/used/',
                'https://auto.ru/belgorodskaya_oblast/cars/bmw/used/',
                'https://auto.ru/kurskaya_oblast/cars/bmw/used/'
                ]

# car_page_templates = [f'https://auto.ru/moskva/cars/bmw/used/?page={PAGE_SELECTOR}&output_type=list',
#                       f'https://auto.ru/sankt-peterburg/cars/bmw/used/?page={PAGE_SELECTOR}&output_type=list']
car_page_templates = [x+f'?page={PAGE_SELECTOR}&output_type=list' for x in regions_urls]

CARS_A_LIST=[]
RANDOM_SEED = 20726
CURRENT_DATE = pd.to_datetime('01/01/2020')
PATH_TO_LINKS_FILE = 'C:/Users/xxx/SF/SkillFactory/car_links_04.txt'
PATH_TO_JSON_FILE = 'C:/Users/xxx/SF/SkillFactory/car_links_cars_04.txt'
dir_name = 'C:/Users/xxx/SF/SkillFactory/data/kagle/sf-dst-auto'
file_name = 'test.csv'
path_to_dataset = os.path.join(dir_name, file_name)

In [4]:
def get_requested_soup_by_url(url, proxiesIN=[], n=3, debug=False):
    '''
    Супчик по url
    Можно использовать список прокси [{'IP': '206.127.88.18', 'Port': 80}, ...]
    Делает 3 попытки для соединения без прокси и количество попыток с прокси равное к-ву прокси + 3 без прокси
    '''
    if debug: print('N-->', n)
    if n<=0:   
        return BeautifulSoup('', 'html.parser')

    if (len(proxiesIN) > 0): #если работаем через прокси
        i = random.randint(0, len(proxiesIN)-1)
        prox = proxiesIN[i]['IP'] + ':' + str(proxiesIN[i]['Port'])
        proxies = {
            'http': 'http://' + prox,
            'https': 'http://' +prox,
            }

        try:
            if debug: print('-->', 0)
            response = requests.get(url, headers=headers, proxies=proxies, timeout=(1, 3))
            if debug: print('-->', 1)
            if response.status_code != 200:
                if debug: print('-->', 2)
                time.sleep(.5)
                return get_requested_soup_by_url(url,proxiesIN, n=n-1)               
        except:
            if debug: print('-->', 4)
            del(proxiesIN[i])
            time.sleep(.5)
            return get_requested_soup_by_url(url,proxiesIN) 
              
    else: #если работаем без прокси
        try:
            if debug: print('-->', 6)
            time.sleep(.5)
            response = requests.get(url, headers=headers, timeout=(1, 3))
            if response.status_code != 200:
                time.sleep(.5)
                if debug: print('-->', 7)
                return get_requested_soup_by_url(url, n=n-1)
                
        except:
            if debug: print('-->', 9)
            time.sleep(.5)
            return get_requested_soup_by_url(url, n=n-1) 

    if debug: print('-->', 10)  
    response.encoding = 'utf-8'
    
    return BeautifulSoup(response.text, 'html.parser')
    #----------------------------------------------------------------------------------
      

def is_port_open(ip,port):
    '''
    Проверяет открыт ли порт - используется для первичной проверки доступности proxy
    '''
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        s.connect((ip, int(port)))
        s.shutdown(2)
        return True
    except Exception as e:
            print('Ошибка:\n', traceback.format_exc())
            return False
#----------------------------------------------------------------------------------
    
def get_free_proxies(top=10):
    '''
    Возвращает список активных free proxy
    По умолчанию проверяем top 10, вернуть может меньше
    [{'IP': '206.127.88.18', 'Port': 80}]
    '''
    tested_proxies=[]
    resp = requests.get('https://scrapingant.com/free-proxies/', headers=headers) 
    proxies = pd.read_html(resp.text)[0]
    proxies = proxies[proxies.Protocol == 'HTTP'].head(top)
    proxies = proxies[['Ip', 'Port' ]]
    for proxy in tqdm(proxies.iterrows()):

        if is_port_open(proxy[1]['Ip'], proxy[1]['Port']) == True:          
            tested_proxies.append({'IP' : proxy[1]['Ip'], "Port" : proxy[1]['Port']})

    return tested_proxies
#----------------------------------------------------------------------------------
    
def get_pages_count(cars_pageN_soup):
    '''
    Возвращает количество страниц объявлений по запросу
    Принимает soup по по URL
    '''
    pages_count=0
    list_of_pages = cars_pageN_soup.find_all('a', href=True, class_='Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination-module__page')

    if len(list_of_pages) == 0:
        pages_count = 1
    else:
        pages_count = int(list_of_pages[-1].find('span', class_='Button__text').text)
        
    return pages_count  
#----------------------------------------------------------------------------------
    
def get_cars_A_from_page(URL, page_number, proxies):
    '''
    Получаем список ссылок на объявления на page_number-странице, по шаблону URL 
    '''
    page_cars_A =[]
    url = URL.replace(PAGE_SELECTOR,str(page_number))
    #print('url--->', url)
    pageN = get_requested_soup_by_url(url, proxies, debug=False)
    cars_a = pageN.find_all('a', href=True, class_='Link ListingItemThumb')

    for car_a in cars_a:
            page_cars_A.append(car_a['href'])

    return page_cars_A
#----------------------------------------------------------------------------------
    
def get_cars_A_all(proxies_in=[]):
    '''
    Получаем все ссылки на объявления 
    '''
    cars_A_all = []
    cars_pages=0
    for cityN in tqdm(range(0, len(regions_urls))):
        if cityN ==0:
            cars_pages = 146 #затычка для Москвы
        else:
            #time.sleep(.2)
            cars_pages = get_pages_count(get_requested_soup_by_url(regions_urls[cityN], proxies_in, debug=False))
        print('------')
        print('city-->',cityN, 'cars_pages-->',cars_pages)
        for pageN in tqdm(range(1, cars_pages+1)):
            page_cars_A = get_cars_A_from_page(car_page_templates[cityN],pageN, proxies_in)
            
            cars_A_all.extend(page_cars_A)
            print('page_cars_A-->', len(page_cars_A), 'cars_A_all-->', len(cars_A_all) )

    return cars_A_all  

In [5]:
'''
Карточка автомобиля
'''

class Car(object):
   
    def __init__(self, brand, productionDate, modelDate, mileage, bodyType, color, config, tax
                        ,vehicleTransmission, driving_gear, driving_side, condition, ownersN, PTS, customs_status
                        ,VIN, vehicleConfiguration, engineDisplacement, enginePower, fuelType
                        ,name, description, options, price,numberOfDoors, url_in, owningTime ):
        self.brand = brand
        self.productionDate = productionDate
        self.modelDate = modelDate
        self.mileage = mileage
        self.bodyType = bodyType
        self.color = color
        self.config = config
        self.tax = tax
        self.vehicleTransmission = vehicleTransmission
        self.driving_gear = driving_gear
        self.driving_side = driving_side
        self.condition = condition
        self.ownersN = ownersN
        self.PTS = PTS
        self.customs_status = customs_status
        self.VIN = VIN
        self.vehicleConfiguration = vehicleConfiguration
        self.engineDisplacement = engineDisplacement
        self.enginePower = enginePower
        self.fuelType = fuelType
        self.name = name
        self.description = description
        self.options = options
        self.price = price
        self.numberOfDoors = numberOfDoors
        self.url = url
        self.owningTime = owningTime
#----------------------------------------------------------------------------------
    
    def to_dict(self):
        '''
        Получаем карточку автомобиля в виде подобном test
        '''
        return {
            
            'bodyType': self.bodyType,
            'brand': self.brand,
            'color': self.color,
            'fuelType': self.fuelType,
            'modelDate': self.modelDate,
            'name': self.name,
            'numberOfDoors': self.numberOfDoors,
            'productionDate': self.productionDate,
            'vehicleConfiguration': self.vehicleConfiguration,
            'vehicleTransmission': self.vehicleTransmission,
            'engineDisplacement': self.engineDisplacement,
            'enginePower': self.enginePower,
            'description': self.description,
            'mileage': self.mileage,
            'Комплектация': self.options,
            'Привод': self.driving_gear,
            'Руль': self.driving_side,
            'Состояние': self.condition,
            'Владельцы': self.ownersN,
            'ПТС': self.PTS,
            'Таможня': self.customs_status,
            'Владение': self.owningTime,
            'id': np.nan,
            'url': self.url,
            'price': self.price
        }

In [6]:

class Car_Reader(object):
    '''
    Класс получения карточки автомобиля с прокси или без (передать пустой список)
    '''

    def __init__(self, car_url, proxies):
        self.url = car_url
        self.proxies = proxies
        self.retry_later_url = ''
    #----------------------------------------------------------------------------------
        
    def _toInt(self, s):
        '''
        Строка в int, убираем &nbsp; в числе, например 8 500->8500
        '''
        val = np.nan
        try: 
            s = s.replace('\xa0','')
            val = int(s)
        except Exception as e:
            print('Ошибка:\n', traceback.format_exc())
        return s#val
    #----------------------------------------------------------------------------------

    def _str2int(self, s_in):
        '''
        Получаем число из строки,отбросив текст 
        '''
        s=''
        try:
            s = s_in.replace('\xa0','').replace('км','').replace('₽ / год','').replace('₽','')
        except Exception as e:
            print('Ошибка:\n', traceback.format_exc())
            return np.nan
        return self._toInt(s)
    #----------------------------------------------------------------------------------

    def _parse_description(self, descr):
        '''
        Получаем описание автомобиля в виде одной стоки 
        '''
        desciption=''
        try:
            descr_soup = descr.contents
            descr_items = len(descr_soup)
            descr.contents
            for i in range(0, descr_items):
                desciption += str(descr_soup[i]).replace('<br/>', ' ')
        except:
            pass
        return desciption
    #----------------------------------------------------------------------------------

    def _get_car_options_packs(self, car_page):
        '''
        Получаем список опциональных пакетов и их содержимое 
        '''
        options_packs_list=[]
        try:
            options_packs = ''
            options_packs_count = len(car_page.find_all('div')[1].find('div', class_='content').find_all('div', class_='CardComplectation__itemContentWrap'))
            options_packs_block = car_page.find_all('div')[1].find('div', class_='content').find_all('div', class_='CardComplectation__itemContentWrap')
            
            for pack_indx in range(0, options_packs_count):
                pack_name = options_packs_block[pack_indx].parent.find('span', class_='CardComplectation__itemName').text
                pack_option_dict = {'name': pack_name,
                                    'values': []}
                pack_option_list = []

                options_count = len(options_packs_block[pack_indx].find_all('li', class_='CardComplectation__itemContentEl'))
                for option_indx in range(0, options_count):
                    pack_option_list.append(options_packs_block[pack_indx]
                                            .find_all('li', class_='CardComplectation__itemContentEl')[option_indx]
                                            .text)

                pack_option_dict['values'] = pack_option_list
                options_packs_list.append(pack_option_dict)
                options_packs = str(options_packs_list).replace("'", '"')
                options_packs = "[\'"+ options_packs + "\']"
        except:
            pass #проглотим исключение, с ним нечего делать

        return options_packs
    #----------------------------------------------------------------------------------

    def _safe_run(self, str, car_page_in):
        '''
        Функция-обертка для "проглатывания" исключений.
        Не все поля у объявления могут быть заполнены, чтоб не терять все объявление оборачиваем в обертку каждое поле.
        '''
        car_page = car_page_in
        try:
            return eval(str)
        except Exception as e:
            #traceback.format_exc(),
            print(' Ошибка отработана:\n', '---> ', str, ' <---')
            return np.nan
    #----------------------------------------------------------------------------------
    
    def _metadata_to_dict(self, meta):
        '''
        Преобразование метадатнных в словарь
        '''
        data={}
        for m in meta:
            try:
                data[m.get('itemprop')] = m.get('content')
            except:
                pass
        return data
    #---------------------------------------------------------------------------------- 
      
    def get_car_data(self, retries=2 ):
        '''
        Получаем все характеристики авто из объявления
        Возвращает класс Car описывающий машину
        '''
        
        url_in = self.url.strip()
        proxies_in = self.proxies
        car = Car(np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
                    ,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
                    ,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
                    ,np.nan, np.nan, np.nan)        

        try:

            car_page = get_requested_soup_by_url(url_in, proxies_in)
            if car_page == None: return car

            metadata = self._metadata_to_dict(car_page.find('span', itemscope='', itemtype='http://schema.org/Car').find_all('meta'))
            #time.sleep(.5)
            
            productionDate = metadata.get('productionDate', '')

            brand = metadata.get('brand', '')

            mileage = self._safe_run(
                "self._str2int(car_page.find('li', class_='CardInfo__row CardInfo__row_kmAge').find_all('span', class_='CardInfo__cell')[1].text)"
                , car_page)

            bodyType = metadata.get('bodyType', '')

            numberOfDoors = metadata.get('numberOfDoors', '') 

            color = metadata.get('color', '')

            config = metadata.get('vehicleConfiguration', '')

            tax = self._safe_run(
                "self._str2int(car_page.find('li', class_='CardInfo__row CardInfo__row_transportTax').find_all('span', class_='CardInfo__cell')[1].text)"
                , car_page)

            vehicleTransmission = metadata.get('vehicleTransmission', '')

            driving_gear = self._safe_run(
                "car_page.find('li', class_='CardInfo__row CardInfo__row_drive').find_all('span', class_='CardInfo__cell')[1].text"
                , car_page)

            driving_side = self._safe_run(
                "car_page.find('li', class_='CardInfo__row CardInfo__row_wheel').find_all('span', class_='CardInfo__cell')[1].text"
                , car_page)

            condition = self._safe_run(
                "car_page.find('li', class_='CardInfo__row CardInfo__row_state').find_all('span', class_='CardInfo__cell')[1].text"
                , car_page)

            ownersN = self._safe_run(
                "car_page.find('li', class_='CardInfo__row CardInfo__row_ownersCount').find_all('span', class_='CardInfo__cell')[1].text"
                , car_page)

            PTS = self._safe_run(
                "car_page.find('li', class_='CardInfo__row CardInfo__row_pts').find_all('span', class_='CardInfo__cell')[1].text"
                , car_page)

            owningTime = self._safe_run(
                "car_page.find('li', class_='CardInfo__row CardInfo__row_owningTime').find_all('span', class_='CardInfo__cell')[1].text"
                , car_page)

            customs_status = self._safe_run(
                "car_page.find('li', class_='CardInfo__row CardInfo__row_customs').find_all('span', class_='CardInfo__cell')[1].text"
                , car_page)

            VIN = self._safe_run(
                "car_page.find('li', class_='CardInfo__row CardInfo__row_vin').find_all('span', class_='CardInfo__cell')[1].text[0:3]"
                , car_page)

            vehicleConfiguration = metadata.get('vehicleConfiguration', '') 

            if car_page.find_all('div')[1].find('div', class_='content').find('div', class_='CardComplectation__titleWrap') is not None:
                vehicleConfiguration = self._safe_run(
                    "car_page.find_all('div')[1].find('div', class_='content').find('div', class_='CardComplectation__titleWrap').text.replace('Комплектация','')"
                    , car_page)

            engineDisplacement = metadata.get('engineDisplacement', '')

            enginePower = metadata.get('enginePower', '')

            modelDate = metadata.get('modelDate', '')

            fuelType = metadata.get('fuelType', '')

            name = self._safe_run(
                "car_page.find_all('h1')[0].find_all('meta', itemprop='name')[-1].get('content')"
                , car_page)
            
            options =self._get_car_options_packs(car_page)

            image = metadata.get('image', '')

            price = metadata.get('price', '')

            description = metadata.get('description', '')
            # description1 = self._parse_description(
            #     self._safe_run(
            #         "car_page.find_all('div')[1].find('div', class_='content').contents[1].find('div', class_='CardDescription__textInner').contents[0]"
            #         , car_page))

            car = Car(brand, productionDate, modelDate, mileage, bodyType, color, config, tax
                        ,vehicleTransmission, driving_gear, driving_side, condition, ownersN, PTS, customs_status
                        ,VIN, vehicleConfiguration, engineDisplacement, enginePower, fuelType
                        ,name, description,  options,   price,numberOfDoors, url_in, owningTime)

        except Exception as e:
            print('Ошибка:\n', traceback.format_exc())
            self.retry_later_url = url_in.strip()

        finally:
            self.retry_later_url = ''
            return car


In [7]:
class Scrapper(threading.Thread):
    '''
    Загружаем ПАРАЛЛЕЛЬНО несколько карточек
    '''
    def __init__(self, threadId, name, url, proxies_in):
        threading.Thread.__init__(self)
        self.name = name
        self.id = threadId
        self.url = url
        self.proxies = proxies_in
    def run(self):
        # Пример закомментирован и специально оставлен! 
        # r = requests.get(self.url)
        # soup = BeautifulSoup(r.content, 'html.parser')
        # links = soup.find_all("a")

        car = Car_Reader(url, proxies)
        res = car.get_car_data().to_dict()
        return res

### кэшируем проверенные на данный момент живые прокси  

In [8]:
proxies = get_free_proxies(50)# прокси кэшируем


2it [00:21, 10.53s/it]Ошибка:
 Traceback (most recent call last):
  File "<ipython-input-4-570a2d25976d>", line 61, in is_port_open
    s.connect((ip, int(port)))
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

6it [00:42,  9.98s/it]Ошибка:
 Traceback (most recent call last):
  File "<ipython-input-4-570a2d25976d>", line 61, in is_port_open
    s.connect((ip, int(port)))
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

10it [01:21,  9.82s/it]Ошибка:
 Traceback (most recent call last):
  File "<ipython-input-4-570a2d25976d>", line 61, in is_port_open
    s.connect((ip, int(port)))
TimeoutError: [WinError 10060] A connection attempt failed because the connected

In [10]:
len(proxies)

17

In [10]:
CARS_A_LIST = get_cars_A_all(proxies) #собираем все ссылки на автомобли
CARS_A_LIST = list(set(CARS_A_LIST)) #уникальные ссылки
with open(PATH_TO_LINKS_FILE, 'w') as f:
    for item in CARS_A_LIST:
        f.write("%s\n" % item)

--> 38 cars_A_all--> 4255

 86%|████████▋ | 126/146 [10:06<01:22,  4.15s/it]page_cars_A--> 38 cars_A_all--> 4293

 87%|████████▋ | 127/146 [10:11<01:25,  4.50s/it]page_cars_A--> 38 cars_A_all--> 4331

 88%|████████▊ | 128/146 [10:16<01:19,  4.44s/it]page_cars_A--> 38 cars_A_all--> 4369

 88%|████████▊ | 129/146 [10:19<01:11,  4.21s/it]page_cars_A--> 38 cars_A_all--> 4407

 89%|████████▉ | 130/146 [10:23<01:04,  4.03s/it]page_cars_A--> 38 cars_A_all--> 4445

 90%|████████▉ | 131/146 [10:27<01:01,  4.12s/it]page_cars_A--> 38 cars_A_all--> 4483

 90%|█████████ | 132/146 [10:32<01:01,  4.42s/it]page_cars_A--> 38 cars_A_all--> 4521

 91%|█████████ | 133/146 [10:37<00:57,  4.45s/it]page_cars_A--> 38 cars_A_all--> 4559

 92%|█████████▏| 134/146 [10:41<00:51,  4.26s/it]page_cars_A--> 38 cars_A_all--> 4597

 92%|█████████▏| 135/146 [10:44<00:44,  4.02s/it]page_cars_A--> 38 cars_A_all--> 4635

 93%|█████████▎| 136/146 [10:48<00:38,  3.86s/it]page_cars_A--> 37 cars_A_all--> 4672

 94%|█████████▍|

---

In [11]:
websites = []
bmw_cars = []
proxies1 = proxies
with open(PATH_TO_LINKS_FILE, 'r') as f: #car_links-OK.txt
    for line in tqdm(f):
        websites.append(line.strip())

i = 1
for url in tqdm(websites):
    print('Первая попытка: ', url, '\n')
    i+=1
    thread = Scrapper(i, "thread"+str(i), url, proxies1)
    res = thread.run()

    if pd.isnull(res['price']):
        print('Вторая попытка: ', url, '\n')
        i+=1
        thread = Scrapper(i, "thread"+str(i), url, proxies1)
        res = thread.run()

    bmw_cars.append(res)# list.append - потокобезопасный метод 

.ru/cars/used/sale/bmw/3er/1098801702-836d7a34/ 

 99%|█████████▉| 9816/9898 [4:38:40<02:05,  1.54s/it] Ошибка отработана:
 --->  car_page.find('li', class_='CardInfo__row CardInfo__row_owningTime').find_all('span', class_='CardInfo__cell')[1].text  <---
Первая попытка:  https://auto.ru/cars/used/sale/bmw/7er/1087832254-2db7c8a8/ 

 99%|█████████▉| 9817/9898 [4:38:41<02:00,  1.48s/it] Ошибка отработана:
 --->  car_page.find('li', class_='CardInfo__row CardInfo__row_owningTime').find_all('span', class_='CardInfo__cell')[1].text  <---
 Ошибка отработана:
 --->  car_page.find('li', class_='CardInfo__row CardInfo__row_vin').find_all('span', class_='CardInfo__cell')[1].text[0:3]  <---
Первая попытка:  https://auto.ru/cars/used/sale/bmw/5er/1089406408-dfdc1942/ 

 99%|█████████▉| 9818/9898 [4:38:43<01:57,  1.47s/it]Первая попытка:  https://auto.ru/cars/used/sale/bmw/5er/1092298452-2f0e682c/?geo_id=10645 

 99%|█████████▉| 9819/9898 [4:38:44<02:02,  1.56s/it] Ошибка отработана:
 --->  car_pag

In [12]:
bmw_cars

, "Мультифункциональное рулевое колесо", "Прикуриватель и пепельница", "Активный усилитель руля", "Подрулевые лепестки переключения передач", "Электропривод зеркал", "Электростеклоподъёмники задние", "Парктроник передний", "Парктроник задний", "Система «старт-стоп»", "Электростеклоподъёмники передние", "Бортовой компьютер", "Запуск двигателя с кнопки", "Электропривод крышки багажника", "Регулировка руля по вылету", "Регулировка руля по высоте", "Климат-контроль 1-зонный", "Электроскладывание зеркал"]}, {"name": "Безопасность", "values": ["Система стабилизации (ESP)", "Подушка безопасности пассажира", "Подушка безопасности водителя", "Крепление детского кресла (задний ряд) ISOFIX", "Система помощи при старте в гору (HSA)", "Подушки безопасности оконные (шторки)", "Подушки безопасности боковые", "Датчик давления в шинах", "Антиблокировочная система (ABS)", "Блокировка замков задних дверей"]}, {"name": "Обзор", "values": ["Система адаптивного освещения", "Ксеноновые/Биксеноновые фары", "Э

In [13]:
len(bmw_cars)

9898

In [14]:
bmw_cars_clone = bmw_cars.copy()

In [15]:
 with open(PATH_TO_JSON_FILE, 'w') as f:
    json.dump(bmw_cars, f)
 

In [17]:
proxies

[]

---

In [55]:
df_j.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'id', 'url', 'price'],
      dtype='object')

In [54]:
df_j = pd.read_json(PATH_TO_JSON_FILE) 
df_j

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id,url,price
0,седан,BMW,чёрный,бензин,NaN,NaN,4.0,1998.0,None,механическая,...,задний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,None,NaN,https://auto.ru/cars/used/sale/bmw/5er/1088602...,170000.0
1,седан,BMW,чёрный,бензин,NaN,NaN,4.0,2011.0,None,автоматическая,...,полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,8 лет и 2 месяца,NaN,https://auto.ru/cars/used/sale/bmw/7er/1099947...,965000.0
2,седан,BMW,чёрный,бензин,NaN,NaN,4.0,1999.0,None,автоматическая,...,задний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,5 лет и 2 месяца,NaN,https://auto.ru/cars/used/sale/bmw/5er/1096411...,180000.0
3,внедорожник 5 дв.,BMW,синий,бензин,NaN,NaN,5.0,2009.0,,автоматическая,...,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,11 лет и 5 месяцев,NaN,https://auto.ru/cars/used/sale/bmw/x6/11002177...,1300000.0
4,седан,BMW,чёрный,бензин,NaN,NaN,4.0,2008.0,520,автоматическая,...,задний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,None,NaN,https://auto.ru/cars/used/sale/bmw/5er/1099992...,600000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10151,внедорожник 5 дв.,BMW,серый,бензин,NaN,NaN,5.0,2004.0,None,автоматическая,...,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,None,NaN,https://auto.ru/cars/used/sale/bmw/x3/10966846...,490000.0
10152,седан,BMW,синий,бензин,NaN,NaN,4.0,2019.0,530i xDrive M Sport,автоматическая,...,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,None,NaN,https://auto.ru/cars/used/sale/bmw/5er/1099162...,3229000.0
10153,седан,BMW,белый,бензин,NaN,NaN,4.0,2012.0,750,автоматическая,...,полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,None,NaN,https://auto.ru/cars/used/sale/bmw/7er/1094575...,980000.0
10154,седан,BMW,белый,бензин,NaN,NaN,4.0,2012.0,None,автоматическая,...,задний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,None,NaN,https://auto.ru/cars/used/sale/bmw/5er/1095777...,1220000.0


In [120]:
df_j[df_j['Комплектация'] ==''].url

2       https://auto.ru/cars/used/sale/bmw/x5/10964989...
4       https://auto.ru/cars/used/sale/bmw/x5/10971235...
6       https://auto.ru/cars/used/sale/bmw/3er/1094603...
10      https://auto.ru/cars/used/sale/bmw/5er/1099937...
15      https://auto.ru/cars/used/sale/bmw/x6/11002057...
                              ...                        
9683    https://auto.ru/cars/used/sale/bmw/3er/1095706...
9686    https://auto.ru/cars/used/sale/bmw/5er/1099896...
9687    https://auto.ru/cars/used/sale/bmw/3er/1100407...
9689    https://auto.ru/cars/used/sale/bmw/7er/1099783...
9691    https://auto.ru/cars/used/sale/bmw/5er/1100212...
Name: url, Length: 3606, dtype: object

In [83]:
df_j.Таможня.value_counts() #столбец содержит одно значение -- удаляем

Растаможен    9357
Name: Таможня, dtype: int64

In [92]:
len(df_j.url.unique())

9358

In [97]:
df_j.drop(['modelDate', 'name', 'id', 'Таможня' ], inplace=True, axis=1)

In [100]:
df_j.dropna(axis=0, how='all', inplace=True)

In [122]:
df_j.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'numberOfDoors',
       'productionDate', 'vehicleConfiguration', 'vehicleTransmission',
       'engineDisplacement', 'enginePower', 'description', 'mileage',
       'Комплектация', 'Привод', 'Руль', 'Состояние', 'Владельцы', 'ПТС',
       'Владение', 'url'],
      dtype='object')

In [101]:
df_j

,bodyType,brand,color,fuelType,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Владение,url
0,внедорожник 5 дв.,BMW,чёрный,бензин,5.0,2016.0,xDrive35i Luxury,автоматическая,3.0 LTR,306 N12,Продаю свой автомобиль BWM X5 2016 года по ПТС...,79000.0,"['[{""name"": ""Комфорт"", ""values"": [""Круиз-контр...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,3 года и 7 месяцев,https://auto.ru/cars/used/sale/bmw/x5/10981645...
1,седан,BMW,чёрный,бензин,4.0,2011.0,,автоматическая,4.4 LTR,407 N12,Машина в отличном техническом и внешнем состоя...,115000.0,"['[{""name"": ""Салон"", ""values"": [""Тонированные ...",полный,Левый,Не требует ремонта,3 или более,Дубликат,None,https://auto.ru/cars/used/sale/bmw/7er/1095055...
2,внедорожник 5 дв.,BMW,серый,бензин,5.0,2014.0,None,автоматическая,3.0 LTR,306 N12,Два владельца Крашено левое зад крыло и зад дв...,140000.0,,полный,Левый,Не требует ремонта,2 владельца,Оригинал,None,https://auto.ru/cars/used/sale/bmw/x5/10964989...
4,внедорожник 5 дв.,BMW,чёрный,дизель,5.0,2007.0,None,автоматическая,3.0 LTR,235 N12,"На стоянки не ставлю, от машине не избавляюсь,...",250000.0,,полный,Левый,Не требует ремонта,3 или более,Оригинал,None,https://auto.ru/cars/used/sale/bmw/x5/10971235...
5,внедорожник 5 дв.,BMW,чёрный,дизель,5.0,2018.0,,автоматическая,3.0 LTR,249 N12,Действующая КАСКО Обслуживание у официального...,39135.0,"['[{""name"": ""Салон"", ""values"": [""Тонированные ...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,1 год и 10 месяцев,https://auto.ru/cars/used/sale/bmw/x6/10984396...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9687,седан,BMW,чёрный,бензин,4.0,1997.0,None,механическая,1.6 LTR,102 N12,СРОЧНО Продам свой авто. Ездила максимально ак...,280000.0,,задний,Левый,Не требует ремонта,3 или более,Оригинал,None,https://auto.ru/cars/used/sale/bmw/3er/1100407...
9688,внедорожник 5 дв.,BMW,белый,дизель,5.0,2009.0,,автоматическая,3.0 LTR,286 N12,В продаже BMW X6 в отличном состоянии. - 2009 ...,160232.0,"['[{""name"": ""Комфорт"", ""values"": [""Круиз-контр...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,None,https://auto.ru/cars/used/sale/bmw/x6/10999842...
9689,седан,BMW,чёрный,дизель,4.0,2010.0,None,автоматическая,3.0 LTR,306 N12,"В отличном состоянии больше времени стоял , на...",82000.0,,полный,Левый,Не требует ремонта,3 или более,Оригинал,None,https://auto.ru/cars/used/sale/bmw/7er/1099783...
9690,внедорожник 5 дв.,BMW,чёрный,бензин,5.0,2017.0,,автоматическая,2.0 LTR,192 N12,РОЛЬФ - Официальный дилер и Лучший дилер Росси...,19700.0,"['[{""name"": ""Салон"", ""values"": [""Комбинированн...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,None,https://auto.ru/cars/used/sale/bmw/x1/11003379...


In [102]:
design_report = ProfileReport(df_j)
design_report.to_file("car_data_output.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 76.87it/s]


In [650]:
df = pd.read_csv(path_to_dataset)

In [651]:
df.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'id'],
      dtype='object')

In [652]:
pprint(df.description,width=200)

0       В РОЛЬФ Ясенево представлено более 500 автомоб...
1       Автомобиль из демонстрационного парка по спец ...
2                       Машина собиралась под заказ! \n\n
3       АВТОМОБИЛЬ В БЕЗУПРЕЧНОМ ВНЕШНЕМ И ТЕХНИЧЕСКОМ...
4       Автомобиль не имеет собственников ( физ. лиц) ...
                              ...                        
3832    Продаётся BMW 5 F10 Рестайлинг\n2 бережливых в...
3833    ✪  ✪  ✪  ✪  ✪  ✪  ✪  ✪  ✪ ✪  ✪  ✪  ✪  ✪  ✪  ✪ ...
3834    Продам полностью обслуженный автомобиль , сдел...
3835    Продаю отличный авто. Полностью обслужен. Лучш...
3836    Максимальная комплектация. 22.09.2019 сделано ...
Name: description, Length: 3837, dtype: object


In [653]:
df.Комплектация[0]

'[\'[{"name":"Безопасность","values":["Антипробуксовочная система (ASR)","Система стабилизации (ESP)","Подушка безопасности водителя","Крепление детского кресла (задний ряд) ISOFIX","Подушки безопасности оконные (шторки)","Подушка безопасности пассажира","Подушки безопасности боковые задние","Подушки безопасности боковые","Датчик давления в шинах","Антиблокировочная система (ABS)","Блокировка замков задних дверей","Подушка безопасности для защиты коленей водителя"]},{"name":"Комфорт","values":["Система «старт-стоп»","Электростеклоподъёмники передние","Мультифункциональное рулевое колесо","Прикуриватель и пепельница","Бортовой компьютер","Усилитель руля","Электропривод зеркал","Электростеклоподъёмники задние","Парктроник передний","Регулировка руля по вылету","Климат-контроль 2-зонный","Электроскладывание зеркал","Парктроник задний"]},{"name":"Мультимедиа","values":["AUX","USB","Аудиосистема","Bluetooth"]},{"name":"Обзор","values":["Датчик света","Автоматический корректор фар","Омывател

In [654]:
 import ast
 import json
 car_options_packs = json.loads(ast.literal_eval(df.Комплектация[0])[0])

In [655]:
car_options_packs

[{'name': 'Безопасность',
  'values': ['Антипробуксовочная система (ASR)',
   'Система стабилизации (ESP)',
   'Подушка безопасности водителя',
   'Крепление детского кресла (задний ряд) ISOFIX',
   'Подушки безопасности оконные (шторки)',
   'Подушка безопасности пассажира',
   'Подушки безопасности боковые задние',
   'Подушки безопасности боковые',
   'Датчик давления в шинах',
   'Антиблокировочная система (ABS)',
   'Блокировка замков задних дверей',
   'Подушка безопасности для защиты коленей водителя']},
 {'name': 'Комфорт',
  'values': ['Система «старт-стоп»',
   'Электростеклоподъёмники передние',
   'Мультифункциональное рулевое колесо',
   'Прикуриватель и пепельница',
   'Бортовой компьютер',
   'Усилитель руля',
   'Электропривод зеркал',
   'Электростеклоподъёмники задние',
   'Парктроник передний',
   'Регулировка руля по вылету',
   'Климат-контроль 2-зонный',
   'Электроскладывание зеркал',
   'Парктроник задний']},
 {'name': 'Мультимедиа',
  'values': ['AUX', 'USB', '

In [656]:
df.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'id'],
      dtype='object')

In [657]:
df

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id
0,седан,BMW,чёрный,дизель,2016.0,520d 2.0d AT (190 л.с.),4.0,2017.0,SEDAN AUTOMATIC 2.0,автоматическая,...,158836.0,"['[{""name"":""Безопасность"",""values"":[""Антипробу...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,0
1,седан,BMW,белый,дизель,2018.0,318d 2.0d AT (150 л.с.),4.0,2019.0,SEDAN AUTOMATIC 2.0,автоматическая,...,10.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,1
2,седан,BMW,синий,бензин,2009.0,550i xDrive 4.4 AT (407 л.с.) 4WD,4.0,2012.0,SEDAN AUTOMATIC 4.4,автоматическая,...,120000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,7 лет и 2 месяца,2
3,внедорожник 5 дв.,BMW,белый,дизель,2014.0,30d 3.0d AT (249 л.с.) 4WD,5.0,2015.0,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,...,111466.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,3
4,внедорожник 5 дв.,BMW,синий,дизель,2014.0,M50d 3.0d AT (381 л.с.) 4WD,5.0,2019.0,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,...,11891.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3832,седан,BMW,чёрный,бензин,2013.0,520i 2.0 AT (184 л.с.),4.0,2013.0,SEDAN AUTOMATIC 2.0,автоматическая,...,98000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",задний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,3832
3833,внедорожник 5 дв.,BMW,чёрный,дизель,2013.0,30d 3.0d AT (249 л.с.) 4WD,5.0,2016.0,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,...,169615.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,3833
3834,седан,BMW,чёрный,бензин,2002.0,525xi 2.5 AT (218 л.с.) 4WD,4.0,2006.0,SEDAN AUTOMATIC 2.5,автоматическая,...,180000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,1 год и 7 месяцев,3834
3835,седан,BMW,коричневый,бензин,2009.0,528i xDrive 2.0 AT (245 л.с.) 4WD,4.0,2012.0,SEDAN AUTOMATIC 2.0,автоматическая,...,137500.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,6 лет и 10 месяцев,3835
